In [ ]:
import random, time, sys, json, os
random.seed(time.time())

# ================== 一处集中配置 ==================
# 项目路径
SIMWORLD_DIR      = r"D:\Projects\Food-Delivery-Bench\SimWorld"
LLM_DELIVERY_DIR  = r"D:\Projects\Food-Delivery-Bench\LLM-Delivery"

# 数据文件
ROADS_JSON        = r"D:\Projects\Food-Delivery-Bench\Test_Data\maps\medium-city-20roads\roads.json"
WORLD_JSON        = r"D:\Projects\Food-Delivery-Bench\Test_Data\maps\medium-city-20roads\progen_world_enriched.json"

# 显示与导出总开关
SHOW_BUS              = True
SHOW_BUILDING_LINKS   = True          # 辅助边开关（POI→dock 始终绘制；普通building→dock 由 plain_mode 决定）
PLAIN_BUILDINGS_MODE  = "all"         # "none" | "pudo" | "all"
#  - none: 不画普通灰建筑；不画普通building→dock
#  - pudo: 仅画 PU/DO 的普通灰建筑；仅画它们→dock（导出也是此策略）
#  - all : 画全部普通灰建筑；并画全部普通building→dock

# 订单数量
NUM_ORDERS            = 4

# —— 路名相关 —— 
SHOW_ROAD_NAMES_WINDOW = True
SHOW_ROAD_NAMES_EXPORT = True
ROAD_NAME_FMT          = "{name}"       # 控制台打印格式；图上的路名按 11L/14R 方案

# —— 导出相关（点击“Save PNGs”时生效）——
EXPORT_TO_DISK     = True
EXPORT_DIR         = r"."
EXPORT_BASENAME    = "map"

# —— 控制台打印地图骨架参数 —— 
INCLUDE_CROSSWALKS = True
MAX_EDGES_PRINT    = 200

# —— 动画/路由参数 —— 
AGENT_SPEED_CM_S   = 10000.0
ROUTE_SNAP_CM      = 120.0
# ===================================================

# === 你的项目路径（保持不变） ===
sys.path.insert(0, SIMWORLD_DIR)
sys.path.insert(0, LLM_DELIVERY_DIR)

from PyQt5.QtWidgets import QApplication
from Base.Map import Map
from utils.map_debug_viewer import MapViewer

import utils.map_debug_viewer as mdv
print(mdv.__file__)
print(mdv.MapDebugViewer.draw_map)


# ----------------- 辅助函数 -----------------
def _load_world_nodes(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f).get("nodes", [])


def _xy_from_props(p):
    loc = p.get("location", {})
    return float(loc.get("x", 0.0)), float(loc.get("y", 0.0))


def _sample_restaurant_hints(nodes, k):
    """放回抽样：允许多个订单命中同一家餐馆"""
    cands = [
        n.get("properties", {})
        for n in nodes
        if (n.get("properties", {}).get("poi_type") or n.get("properties", {}).get("type", "")).lower() == "restaurant"
    ]
    if not cands:
        return [None] * k
    result = []
    for _ in range(k):
        props = random.choice(cands)
        x, y = _xy_from_props(props)
        result.append({"x": x, "y": y})
    return result


def _sample_building_hints(nodes, k):
    """放回抽样：允许多个订单命中同一栋楼"""
    cands = [n.get("properties", {}) for n in nodes if str(n.get("instance_name", "")).startswith("BP_Building")]
    if not cands:
        return [None] * k
    result = []
    for _ in range(k):
        props = random.choice(cands)
        x, y = _xy_from_props(props)
        result.append({"x": x, "y": y})
    return result


def _rand_free_xy(m: Map):
    mnx, mxx, mny, mxy = m.bbox()
    pad = 1000.0
    return random.uniform(mnx + pad, mxx - pad), random.uniform(mny + pad, mxy - pad)


# ===== 生成“地图骨架”文本（交叉口 + 合并后边，带路名），用于打印给 Agent/调试 =====
def _map_text(m: Map, include_crosswalks: bool = True, max_edges: int = 200, name_tmpl: str = "{name}") -> str:
    agent_graph = m.export_agent_graph(include_crosswalks=include_crosswalks, print_preview=False)

    # 交叉口坐标映射
    id2xy = {}
    for nd in m.nodes:
        if getattr(nd, "type", "") == "intersection":
            id2xy[str(nd)] = (int(round(nd.position.x)), int(round(nd.position.y)))

    # Intersections
    inters_sorted = sorted(
        (
            id2xy.get(n["id"], (None, None))
            for n in agent_graph.get("nodes", [])
            if n.get("type") == "intersection"
        ),
        key=lambda xy: (
            xy[0] if xy[0] is not None else 10**18,
            xy[1] if xy[1] is not None else 10**18,
        ),
    )
    lines = []
    lines.append(f"Intersections ({len(inters_sorted)}):")
    for xy in inters_sorted:
        if xy[0] is None:
            continue
        lines.append(f"  ({xy[0]}, {xy[1]})")

    # Edges
    def _edge_sort_key(e):
        kind_rank = 0 if (e.get("kind") == "road") else 1
        return (kind_rank, e.get("name", ""), e.get("u", ""), e.get("v", ""))

    edges = sorted(agent_graph.get("edges", []), key=_edge_sort_key)
    lines.append(f"\nEdges ({len(edges)}):")
    for i, e in enumerate(edges):
        if i >= max_edges:
            lines.append(f"  ... ({len(edges) - i} more)")
            break
        ux, uy = id2xy.get(e["u"], (None, None))
        vx, vy = id2xy.get(e["v"], (None, None))
        if ux is None or vx is None:
            continue
        label = e.get("name") or ("crosswalk" if e.get("kind") != "road" else "")
        dist_m = round(float(e.get("dist_cm", 0.0)) / 100.0, 1)
        lines.append(f"  {label}: ({ux}, {uy}) - ({vx}, {vy}) • {dist_m}m")
    return "\n".join(lines)


# ----------------- 地图与订单 -----------------
def build_map():
    m = Map()
    m.import_roads(ROADS_JSON)
    m.import_pois(WORLD_JSON)
    return m


def make_order(m: Map, k: int = NUM_ORDERS):
    nodes = _load_world_nodes(WORLD_JSON)
    pu_hints = _sample_restaurant_hints(nodes, k)
    do_hints = _sample_building_hints(nodes, k)

    orders = []
    for i in range(k):
        orders.append(
            {
                "id": f"{i + 1}",
                "pickup_hint": pu_hints[i],
                "dropoff_hint": do_hints[i],
            }
        )
    m.set_active_orders(orders)


# ================= 主程序 =================
def main():
    app = QApplication(sys.argv)

    m = build_map()
    make_order(m, k=NUM_ORDERS)

    v = MapViewer("Map Viewer")
    v.draw_map(
        m,
        WORLD_JSON,
        show_bus=SHOW_BUS,
        show_building_links=SHOW_BUILDING_LINKS,
        show_road_names=SHOW_ROAD_NAMES_WINDOW,
        road_name_fmt=ROAD_NAME_FMT,  # 控制台打印时生效
        plain_mode=PLAIN_BUILDINGS_MODE,  # ★ 新增：普通 building 显示模式
    )
    v.mark_orders(m)

    # ---------- 刷新 “next hop/next intersection/all POIs” ----------
    def _refresh_agent_brief(ax: float, ay: float):
        pkg = m.agent_info_package_xy(ax, ay)
        map_txt = _map_text(
            m,
            include_crosswalks=INCLUDE_CROSSWALKS,
            max_edges=MAX_EDGES_PRINT,
            name_tmpl=ROAD_NAME_FMT,
        )
        print("\n=== Map Skeleton (compact) ===")
        print(map_txt)
        print("\n=== Agent Brief ===")
        print(pkg["text"])

        v.info.setText(pkg["text"])
        v.highlight_frontier(pkg["reachable"])

    # 初始 agent 坐标
    ax, ay = _rand_free_xy(m)
    v.set_agent_xy(ax, ay)
    _refresh_agent_brief(ax, ay)

    # ---------- 点击 “Go to (x,y)” ----------
    def on_go_to_xy(tx: float, ty: float):
        nonlocal ax, ay
        route = m.route_xy_to_xy(ax, ay, tx, ty, snap_cm=ROUTE_SNAP_CM)
        v.clear_path_highlight()
        v.highlight_path(route)
        v.prepare_animation(route, speed_cm_s=AGENT_SPEED_CM_S)
        v.start_animation()

    v.set_go_callback(on_go_to_xy)

    # ---------- 动画结束 ----------
    def on_anim_done(final_x: float, final_y: float):
        nonlocal ax, ay
        ax, ay = final_x, final_y
        v.draw_map(
            m,
            WORLD_JSON,
            show_bus=SHOW_BUS,
            show_building_links=SHOW_BUILDING_LINKS,
            show_road_names=SHOW_ROAD_NAMES_WINDOW,
            road_name_fmt=ROAD_NAME_FMT,
            plain_mode=PLAIN_BUILDINGS_MODE,  # ★ 记得传入
        )
        v.mark_orders(m)
        _refresh_agent_brief(ax, ay)

    v.set_anim_done_callback(on_anim_done)

    # ---------- Reinit ----------
    def reinit():
        nonlocal ax, ay
        make_order(m, k=NUM_ORDERS)
        v.draw_map(
            m,
            WORLD_JSON,
            show_bus=SHOW_BUS,
            show_building_links=SHOW_BUILDING_LINKS,
            show_road_names=SHOW_ROAD_NAMES_WINDOW,
            road_name_fmt=ROAD_NAME_FMT,
            plain_mode=PLAIN_BUILDINGS_MODE,  # ★ 记得传入
        )
        v.mark_orders(m)
        ax, ay = _rand_free_xy(m)
        v.set_agent_xy(ax, ay)
        _refresh_agent_brief(ax, ay)

    v.set_reinit_callback(reinit)

    # ---------- Save PNGs ----------
    def on_save_pngs():
        pkg = m.agent_info_package_xy(ax, ay)
        reachable = pkg["reachable"]

        ts = time.strftime("%Y%m%d_%H%M%S")
        g_path = os.path.join(EXPORT_DIR, f"{EXPORT_BASENAME}_global_{ts}.png") if EXPORT_TO_DISK else None
        l_path = os.path.join(EXPORT_DIR, f"{EXPORT_BASENAME}_local_{ts}.png") if EXPORT_TO_DISK else None

        if EXPORT_TO_DISK:
            v.save_images(
                reachable,
                global_path=g_path,
                local_path=l_path,
                show_road_names=SHOW_ROAD_NAMES_EXPORT,
            )
            print(f"[Export] Saved to {g_path}, {l_path}")
        else:
            g_bytes, l_bytes = v.generate_images(
                reachable,
                show_road_names=SHOW_ROAD_NAMES_EXPORT,
            )
            print(f"[Export] Returned bytes - global: {len(g_bytes)}, local: {len(l_bytes)}")

    try:
        v.btn_save.clicked.disconnect()
    except Exception:
        pass
    v.btn_save.clicked.connect(on_save_pngs)

    v.show()
    sys.exit(app.exec_())


if __name__ == "__main__":
    main()
